In [1]:
import os
import polars as pl
import altair as alt

import warnings
from src.kristi_promin import kristi_promin
from src.me_to_neurazi import me_to_neurazi
from src.alt_friendly import alt_friendly

pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [2]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).join(
    pl.scan_parquet(
        "data/projevy_jazyky.parquet",
    ),
    how="left",
    on="text"
).filter(
    pl.col("jazyk") != "sk"
).collect()

In [3]:
def vyvoj_vyskytu(hledani, popisek):
    
    return df.with_columns(
        pl.col("text").str.count_matches(hledani).alias("vyskytu")
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("vyskytu").sum()
    ).join(
    df.group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("pocet_slov").sum()
    ),
    how="left",
    on="datum"
).with_columns(
    (pl.col("vyskytu") / pl.col("pocet_slov")).alias("pomer")
).with_columns(
    pl.lit(popisek).alias("co")
)

In [4]:
def graf(frejm):
    return alt.Chart(
    frejm.to_pandas(),
    width=800,
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q"),
    alt.Row("co:N")
)
    

In [33]:
barvy = pl.concat(
        [
            vyvoj_vyskytu(r'[^\w]rud[áýáíěoaé]\w{0,3}[^\w]','rudá'),
            vyvoj_vyskytu(r'[^\w]červen[áýáíěoaé]\w{0,3}[^\w]','červená'),
            vyvoj_vyskytu(r'[^\w]zelen[áýíěoaé]\w{0,3}[^\w]','zelená'),
            vyvoj_vyskytu(r'[^\w]hněd[áýíěoaé]\w{0,3}[^\w]','hnědá'),
            vyvoj_vyskytu(r'[^\w]čern[áýíěoaé]\w{0,3}[^\w]','černá'),
            vyvoj_vyskytu(r'[^\w]bíl[áýíěoaé]\w{0,3}[^\w]','bílá'),
            vyvoj_vyskytu(r'[^\w]šed[áýíěoaé]\w{0,3}[^\w]','šedá'),
            # vyvoj_vyskytu(r'(?i)[^\w]růžov','růžová')
        ]
)

In [35]:
poradi=['rudá','červená','zelená','černá','bílá','hnědá','růžová','šedá']

In [37]:
barvy_graf = alt.Chart(
    barvy.to_pandas(),
    width=250,
    height=50,
    title=alt.Title(
        'Barvy zmiňované v poslaneckých projevech',
        subtitle=['Pro vyloučení jmen a názvů (Josef Černý, Červená Voda…)','hledáme pouze zmínky malými písmeny.']
    )
).mark_bar(
    width=2
).encode(
    alt.X("datum:T", title=None),
    alt.Y("pomer:Q", axis=alt.Axis(title=None, orient="right",format="%")),
    alt.Row(
        "co:N", title=None, header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='start', labelOrient='top', labelFont='Asap'),
        sort=poradi
    ),
    alt.Color("co:N", title=None, legend=None, scale=alt.Scale(
        range=['#D6534B','#D6534B','#70871E','#292829','#445B78','#5E2D3A','#9eb8c1','#E09DA3']
    ), sort=poradi
             )
).resolve_axis(
    x="independent"
).configure_view(
    stroke='transparent'
)

barvy_graf

alt.Chart(...)

In [43]:
barvy_graf.save("grafy/barvy_graf.png", scale_factor=3)

In [21]:
import re

def hledej(text, n=3, delka=200, frejm=df, sestupne=False, url=True):
    nalezy = frejm.sort(by="datum", descending=sestupne).filter(pl.col("text").str.contains(text)).head(n).iter_rows()
    for cislo, radek in enumerate(nalezy):
        print(f'Kdo: {radek[0]}')
        print(f'Kde: {radek[1]}')
        if url == True:
            print(f'URL: https://www.psp.cz/eknih/{radek[1].replace('_','/')}')
        print(f'Kdy: {radek[9]}')
        
        # Split the text around the search term
        parts = re.split(text, radek[3])
        
        # Get the part before the search term (safe if empty)
        before = parts[0] if len(parts) > 0 else ""
        # Get the part after the search term (safe if empty)
        after = parts[1] if len(parts) > 1 else ""
        
        # Take up to 200 characters from before and after, but don't break if shorter
        before_snippet = before[-delka:] if len(before) > delka else before
        after_snippet = after[:delka] if len(after) > delka else after
        nalez = re.search(text, radek[3]).group()
        
        uryvek = before_snippet + nalez + after_snippet
        print(f'Co: {uryvek}')
        
        if cislo != 2:  # Changed from 3 to 2 since enumerate starts at 0
            print("***")

In [23]:
hledej("šedé", sestupne=True, n=10)

Kdo: Poslankyně Alena Schillerová
Kde: 2021ps_stenprot_144schuz_s144029.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/144schuz/s144029.htm
Kdy: 2025-06-17
Co: ho trhu s lihovinami nebo úspěchy Daňové Kobry v boji proti daňovým podvodům. Ano, té Daňové Kobry, o které jsme už tři a půl roku neslyšeli ani pípnutí, natož zasyčení. Jistě pochopíte, že renesance šedé ekonomiky a daňové kriminality, kterou přímo způsobil ministr financí Zbyněk Stanjura, důsledně uplatňující pokřivenou filozofií ODS, že řádný výběr daní rovná se šikana podnikatelů, je pro mě neskut
***
Kdo: Poslankyně Alena Schillerová
Kde: 2021ps_stenprot_144schuz_s144028.htm
URL: https://www.psp.cz/eknih/2021ps/stenprot/144schuz/s144028.htm
Kdy: 2025-06-17
Co: hybí ve všech oblastech a regionech. Chybí ve školství, chybí v dopravě, chybí ve zdravotnictví, chybí v sociální oblasti, chybí policistům, chybí seniorům, chybí rodinám s dětmi. Politikou tolerance šedé ekonomiky cash only, zaměstnávání načerno, zrušením elektron

In [29]:
hledej("[^\w]rud[áýáíěoaé]\w{0,3}", sestupne=False, n=10, frejm=df.filter(pl.col("datum").dt.year().is_between(1930,1935)))

Kdo: Posl. Zápotocký
Kde: 1929ns_ps_stenprot_016schuz_s016003.htm
URL: https://www.psp.cz/eknih/1929ns/ps/stenprot/016schuz/s016003.htm
Kdy: 1930-02-17
Co: skutečně povede k odstranění vykořisťovatelů, k tomu, aby dělná a pracující třída byla skutečně učiněna pánem ve svém domě. V popředí těchto zápasů dělnické třídy stojí komunistická strana a revoluční rudé odborové hnutí dělnické ve všech státech a ve všech zemích. Tito činitelé vedou a organisují dělnickou třídu přes všechny persekuce, přes všechen persekuční nástup, který se provádí proti komunistick
***
Kdo: Posl. Zápotocký (pokračuje)
Kde: 1929ns_ps_stenprot_016schuz_s016004.htm
URL: https://www.psp.cz/eknih/1929ns/ps/stenprot/016schuz/s016004.htm
Kdy: 1930-02-17
Co: lnictva v hedvábnických továrnách. . . . (Posl. dr Stránský: Vy prý odpovíte Srbovi, jak to je s tím Kladnem!) Budu beze všeho odpovídat. Když byla stávka v Mor. Ostravě, udělaly komunistická strana a rudé odborové organisace všechno, aby také na Kladensku byla zahá

In [31]:
hledej("[^\w]hněd[áýáíěoaé]\w{0,3}", sestupne=False, n=10, frejm=df.filter(pl.col("datum").dt.year().is_between(1978,1982)))

Kdo: Předseda vlády ČSSR L. Štrougal
Kde: 1976fs_slsn_stenprot_007schuz_s007003.htm
URL: https://www.psp.cz/eknih/1976fs/slsn/stenprot/007schuz/s007003.htm
Kdy: 1978-04-04
Co: skutečnost je o to závaznější, že jejich získávání z dovozu i z vlastních zdrojů je stále dražší. Například úplné vlastní náklady na 1 tunu černého uhlí stouply za poslední čtyři roky téměř o 11 % a u hnědého uhlí téměř o 12 %. Zvýšená pozornost rychlejšímu rozvoji vlastní těžby paliv přináší první výsledky. V loňském roce byla těžba uhlí o 6,8 mil. tun vyšší než v roce 1975. Spolu s úsilím o optimální za
***
Kdo: Předseda vlády ČSSR L. Štrougal
Kde: 1976fs_slsn_stenprot_007schuz_s007004.htm
URL: https://www.psp.cz/eknih/1976fs/slsn/stenprot/007schuz/s007004.htm
Kdy: 1978-04-04
Co:  Přijali jsme k tomu řadu opatření, která se již příznivě projevila, takže některé celostátně důležité úkoly se podařilo částečně plnit, nebo se postupně zajišťují. To platí zejména o výstavbě většiny hnědouhelných lomů a elektráren Po

In [47]:
hledej("[^\w]starorůž", sestupne=False, n=10)

In [49]:
hledej("[^\w]blankyt", sestupne=False, n=10)

Kdo: Poslanec Miloš Kužvart
Kde: 2002ps_stenprot_022schuz_s022090.htm
URL: https://www.psp.cz/eknih/2002ps/stenprot/022schuz/s022090.htm
Kdy: 2003-10-16
Co: oucnosti, ale z pohledu vybavení nutnou ekologickou infrastrukturou, kterou obce potřebují, potřebuje naše země. My nebudeme stavět ty čističky odpadních vod či odstraňovat staré ekologické zátěže pro blankyt očí bruselských byrokratů. Budeme to dělat pro potřebu této země, naší České republiky. Takže právě z toho důvodu věřím, že tento návrh projde úspěšně třetím čtením i Senátem, že bude přijat a že je 
***
Kdo: Poslanec František Navrkal
Kde: 2017ps_stenprot_098schuz_s098140.htm
URL: https://www.psp.cz/eknih/2017ps/stenprot/098schuz/s098140.htm
Kdy: 2021-04-16
Co: jvyšší soud rozhodl o tom, že některé rozhodčí doložky, které soud ale nepřezkoumal, to prostě rozhodla - no přezkoumával je... no tu konkrétní ne. On Nejvyšší soud přezkoumal právě to přezkoumal jako blankytně, řekl: Za určitých podmínek rozhodčí doložky nejsou platný

In [51]:
hledej("[^\w]fuchsio", sestupne=False, n=10)

In [53]:
hledej("[^\w]fialov", sestupne=False, n=10)

Kdo: Posl. dr Branžovský
Kde: 1935ns_ps_stenprot_062schuz_s062009.htm
URL: https://www.psp.cz/eknih/1935ns/ps/stenprot/062schuz/s062009.htm
Kdy: 1936-10-15
Co: axu autodrožkáři, platí se různé taxy. V Anglii tax znamená clo nebo daň. Ale taxu za řád - z toho čiší takový studený cynický fiskalismus, že je to děs. Kde jsme se ocitli s tím krásným heslem, s tou fialovou párou, která zahalovala všechno, oč se jen otřelo slovo demokracie? Půjdeš zaplatíš si - koupíš si! Nevím, kde jsme se ocitli. Měli jsme "Bílého lva" pro cizince. Mnozí z vás se pamatují na podivný
***
Kdo: Posl. dr Neuman (pokračuje)
Kde: 1935ns_ps_stenprot_081schuz_s081007.htm
URL: https://www.psp.cz/eknih/1935ns/ps/stenprot/081schuz/s081007.htm
Kdy: 1937-02-19
Co: Vážení pánové, račte dovoliti, abych zde řekl, že československá vláda není a nebude nikdy ani rudá, ani zelená nebude ani černá, ani fialová, československá vláda bude vždy červenomodro-bílá, poněvadž vláda republiky bude vždy ve státních barvách republiky. T